In [1]:
import tempfile
import formulallm.formula as f
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import register_function
from typing import Annotated
import io
import contextlib

In [2]:
local_llm_config={
    "config_list": [
        {
            "model": "NotRequired", # Loaded with LiteLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [3]:
dsl_path = input("Input path to 4ml file: ")
domain = input("Input the name of the domain: ")
partial_model = input("Input the name of the partial model: ")

In [9]:
code = f.load("./data/MappingExample.4ml")

(Compiled) MappingExample.4ml
0.61s.


In [4]:
code_fixer = ConversableAgent(
    name = "Code Fixer",
    llm_config = local_llm_config,
    human_input_mode="NEVER",
    system_message = '''You are a code fixer. Given a DSL domain and paritial model pair written in formula, which is potentially unsolvable, modify the provided code to make it solvable. 
when suggesting repairs, prioritize modifying the partial model over modifying the domain constraints unless otherwise specified. You will also be provided with messages that might explain why the DSL is unsolvable.
Do not say anything said isn't code, just provide the entire modified code in formula. Make sure you follow the syntax. For example, 

WorkFlow:
1.understand the domain and the constraints.
2.using the constraints from the domain, modify the partial model. Be as abstract as possible. i.e. use variables like x and y instead of fixed numbers
3.reply with MODIFIED CODE ONLY. Do not speak a single word that is not part of the code. if you do so, i will destroy you
'''
)

In [5]:
user_proxy = ConversableAgent(
    name = "User",
    llm_config = local_llm_config,
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
    system_message="You are the user proxy."
    "Please obey the following workflow strictly!"
    "When you receive a message from the fixer,"
    "first call check_syntax(), to load the DSL file,"
    "then call solve(), to solve the partial model,"
    "then call get_result(), to check whether the model is solvable or not."
    "And finally, ask the human in the loop to provide any feedback"
)

In [6]:
made_solve = False
made_load = False
made_extract = False

In [7]:
def check_syntax() -> Annotated[
    str, "The content written in DSL of the file loaded"]:
    global made_load
    made_load = True
    global dsl_path
    code = f.load(dsl_path)
    return code

def solve() -> Annotated[
              str, "The message indicating the result of the solve command, including parsing, task initiation information, and any module-related messages"]:
    global made_solve
    made_solve = True
    global partial_model, domain
    return f.solve(partial_model, "1", f"{domain}.conforms")

def get_result(id: Annotated[str, "The id of the task, started at 0"]) -> Annotated[
              str, "The solution to the model if solvable; otherwise some least unsatisfied core conditions."]:
    global made_extract
    made_extract = True
    return f.extract(id, "0", "0")

In [8]:
def terminate_nested_chat(msg):
    global made_solve, made_load, made_extract
    if made_solve and made_load and made_extract:
        made_solve = made_load = made_extract = False
        return True
    else:
        return False

In [9]:
executor = ConversableAgent(
    name = "Executor",
    llm_config=False,
    human_input_mode="NEVER",
    is_termination_msg=terminate_nested_chat,
    default_auto_reply="Please check if the suggested repairs work."
)

In [10]:
register_function(
        check_syntax,
        caller = user_proxy,
        executor=executor,
        name="check_syntax",
        description="Check the syntax of the DSL by trying to load the file"
    )

register_function(
        solve,
        caller=user_proxy,
        executor=executor,
        name="solve",
        description="Solve the partial model"
)

register_function(
        get_result,
        caller=user_proxy,
        executor=executor,
        name="get_result",
        description='''Get result of the last "solve" task. 
        If the partial model is solvable, will output the solution to the model;
        If the partial model is unsolvable, will output some least unsatisfied core condition
        as a hint for the fixer to suggest some other repairs.
        '''
)

In [11]:
user_proxy.register_nested_chats(
    trigger=code_fixer,
    chat_queue=[
        {
            "sender": executor,
            "recipient": user_proxy,
            "summary_method": "reflection_with_llm",
        }
    ],
)

In [12]:
stdout_buffer = io.StringIO()
stderr_buffer = io.StringIO()

with contextlib.redirect_stdout(stdout_buffer), contextlib.redirect_stderr(stderr_buffer):
    try:
        code = f.load(dsl_path)
        f.solve(partial_model,"1",f"{domain}.conforms")
        f.extract("0","0","0")
    except Exception as e:
        pass

message = stdout_buffer.getvalue()
error_message = stderr_buffer.getvalue()

print("Captured stdout:", message)
print("Captured stderr:", error_message)

Captured stdout: (Compiled) MappingExample.4ml
0.57s.
Parsing text took: 1
Visiting text took: 0
Started solve task with Id 0.
0.24s.
Model not solvable. Unsat core terms below.
Conflicts: Mapping.badMapping 
Conflicts: Mapping.invalidUtilization 

0.01s.

Captured stderr: 


In [13]:
user_proxy.initiate_chat(
    recipient=code_fixer,
    message=f'''Here is the DSL domain-model pair loaded: 
    {code} 
    
    The partial model is unsolvable. 
    Please provide some suggestions to modify the constraints of the domain to make the model solvable.
    
    Here is the messages that you can use as a hint to fix the constraints: 
    {message}''',
    max_turns=2
)

User (to Code Fixer):

Here is the DSL domain-model pair loaded: 
    domain Mapping
{
  Component ::= new (id: Integer, utilization: Real).
  Processor ::= new (id: Integer).
  Mapping   ::= new (c: Component, p: Processor).

  // The utilization must be > 50
  invalidUtilization :- c is Component, c.utilization <= 50.

  badMapping :- p is Processor, 
		s = sum(0.0, { c.utilization |
			       c is Component, Mapping(c, p) }), s > 100.

  conforms no badMapping, no invalidUtilization.
}

partial model pm of Mapping
{
  c1 is Component(0, x).
  c2 is Component(1, y).
  p1 is Processor(0).
  Mapping(c1, p1).
  Mapping(c2, p1).
} 
    
    The partial model is unsolvable. 
    Please provide some suggestions to modify the constraints of the domain to make the model solvable.
    
    Here is the messages that you can use as a hint to fix the constraints: 
    (Compiled) MappingExample.4ml
0.57s.
Parsing text took: 1
Visiting text took: 0
Started solve task with Id 0.
0.24s.
Model not so

ChatResult(chat_id=None, chat_history=[{'content': 'Here is the DSL domain-model pair loaded: \n    domain Mapping\n{\n  Component ::= new (id: Integer, utilization: Real).\n  Processor ::= new (id: Integer).\n  Mapping   ::= new (c: Component, p: Processor).\n\n  // The utilization must be > 50\n  invalidUtilization :- c is Component, c.utilization <= 50.\n\n  badMapping :- p is Processor, \n\t\ts = sum(0.0, { c.utilization |\n\t\t\t       c is Component, Mapping(c, p) }), s > 100.\n\n  conforms no badMapping, no invalidUtilization.\n}\n\npartial model pm of Mapping\n{\n  c1 is Component(0, x).\n  c2 is Component(1, y).\n  p1 is Processor(0).\n  Mapping(c1, p1).\n  Mapping(c2, p1).\n} \n    \n    The partial model is unsolvable. \n    Please provide some suggestions to modify the constraints of the domain to make the model solvable.\n    \n    Here is the messages that you can use as a hint to fix the constraints: \n    (Compiled) MappingExample.4ml\n0.57s.\nParsing text took: 1\nVisi